In [1]:
import os
import sys
import numpy as np
from collections import defaultdict, Counter
import time
from matplotlib import pyplot
import json5
from matplotlib import pyplot as pyplot
from scipy.cluster.hierarchy import dendrogram
import pandas

import importlib
import duhportinf
import duhportinf._grouper
import duhportinf._optimize
import duhportinf._vectorizer
import duhportinf.busdef
import duhportinf.main_portinf as main
from   duhportinf.busdef import BusDef
from   duhportinf._optimize import MatchCost
from   duhportinf import util

%load_ext autoreload
%autoreload 2

In [2]:
block_root = '/ip-block-designs'
bus_spec_rootdir = '/bus-defs/specs'

In [4]:
def get_ports_from_json5(path):
    with open(path) as fin:
        block = json5.load(fin)
    return util.format_ports(block['definitions']['ports'])

In [5]:
pcie_json_path = os.path.join(
    block_root,
    'block-plda-pcie-controller',
    'PLDA_ws.json5',
)
ddr_json_path = os.path.join(
    block_root,
    'block-cadence-ddr3',
    'ddr.json5',
)
nvdla_json_path = os.path.join(
    block_root,
    'block-nvdla',
    'nvdla.json5',
)
ark_json_path = os.path.join(
    block_root,
    'block-ark',
    'ark.json5',
)
pcie_ports  = get_ports_from_json5(pcie_json_path)
ddr_ports   = get_ports_from_json5(ddr_json_path)
nvdla_ports = get_ports_from_json5(nvdla_json_path)
ark_ports   = get_ports_from_json5(ark_json_path)

In [6]:
bus_defs = duhportinf.load_bus_defs(bus_spec_rootdir)

In [25]:
pg, Z, wn = duhportinf.get_port_grouper(pcie_ports)

In [26]:
i_bus_pairings = main._get_bus_pairings(pg, bus_defs)

In [29]:
interfaces = list(sorted(
    [interface for nid, l_fcost, interface, bds in i_bus_pairings],
    reverse=True,
    key=lambda i: i.size,
))
print('num selected interfaces', len(interfaces))
for interface in interfaces[:100]:
    print('interface size:{}, prefix:{}'.format(interface.size, interface.prefix))



num selected interfaces 330
interface size:65, prefix:tl_
interface size:57, prefix:
interface size:52, prefix:p
interface size:51, prefix:pl_
interface size:49, prefix:axi4_sto
interface size:49, prefix:axi4_mstd_
interface size:49, prefix:axi4_mst3_
interface size:49, prefix:axi4_mst2_
interface size:49, prefix:axi4_mst0_
interface size:49, prefix:axi4_mst1_
interface size:48, prefix:axi4_sto
interface size:48, prefix:axi4_slv3_
interface size:48, prefix:axi4_slv2_
interface size:48, prefix:axi4_slv0_
interface size:48, prefix:axi4_slv1_
interface size:48, prefix:axi4_mstd_
interface size:48, prefix:axi4_mst3_
interface size:48, prefix:axi4_mst2_
interface size:48, prefix:axi4_mst0_
interface size:48, prefix:axi4_mst1_
interface size:47, prefix:axi4_slv3_
interface size:47, prefix:axi4_slv2_
interface size:47, prefix:axi4_slv0_
interface size:47, prefix:axi4_slv1_
interface size:47, prefix:axi4_mstd_
interface size:47, prefix:axi4_mst3_
interface size:47, prefix:axi4_mst2_
interface 

In [7]:
pg, Z, wn = duhportinf.get_port_grouper(nvdla_ports)

In [20]:
def get_fcost2_bus_defs(interface, bus_defs):
    return list(sorted([
        (duhportinf._optimize._get_name_fcost2(interface, bd), bd)
        for bd in bus_defs
        ],
        reverse=True,
        key=lambda x: x[0],
    ))

In [24]:
dport = ("nvdla_core2dbb_ar_arready", 1, 1)
for nid, interface in pg.get_initial_interfaces():
    if dport in interface.ports:
        i_bus_defs = main._get_lfcost_bus_defs(interface, list(bus_defs))[:5]
        i_bus_defs_2 = get_fcost2_bus_defs(interface, list(bus_defs))[:5]
        lfcost, lbd = next(iter(i_bus_defs))
        lfcost2, lbd2 = next(iter(i_bus_defs_2))
        print('interface size:{}, prefix:{}'.format(interface.size, interface.prefix))
        print('  - lfcost:', lfcost)
        print('  - driver_type:{}, abstract_type:{}'.format(
            lbd.driver_type,
            lbd.abstract_type,
        ))
        print('  - token membership:', lfcost2)
        print('  - driver_type:{}, abstract_type:{}'.format(
            lbd2.driver_type,
            lbd2.abstract_type,
        ))
    

interface size:50, prefix:nvdla_core2
  - lfcost: 77.70(n:30.85;w:16;d:0)
  - driver_type:master, abstract_type:{'vendor': 'amba.com', 'library': 'AMBA4', 'name': 'ACE_rtl', 'version': 'r0p0_0'}
  - token membership: 0.627906976744186
  - driver_type:master, abstract_type:{'vendor': 'sifive.com', 'library': 'MEM', 'name': 'SPRAM_rtl', 'version': '0.1.0'}
interface size:25, prefix:nvdla_core2dbb_
  - lfcost: 29.99(n:10.50;w:9;d:0)
  - driver_type:master, abstract_type:{'vendor': 'amba.com', 'library': 'AMBA3', 'name': 'AXI_rtl', 'version': 'r2p0_0'}
  - token membership: 0.6050420168067226
  - driver_type:master, abstract_type:{'vendor': 'amba.com', 'library': 'AMBA4', 'name': 'AXI4-Lite_rtl', 'version': 'r0p0_0'}
interface size:6, prefix:nvdla_core2dbb_ar_ar
  - lfcost: 13.30(n:4.65;w:4;d:0)
  - driver_type:master, abstract_type:{'vendor': 'amba.com', 'library': 'AMBA3', 'name': 'ATB_rtl', 'version': 'r2p0_0'}
  - token membership: 0.37333333333333335
  - driver_type:master, abstract_t

In [ ]:
df = pandas.DataFrame(Z)
df = df.astype({0:int, 1:int, 3:int})
df.to_csv('linkage-tree.csv', header=False, index=False)
df = pandas.DataFrame(wire_names)
df.to_csv('wire-names.csv', header=False, index=False)
df.head()

In [ ]:
def debug(ports, dport, bus_abstract_names):
    # filter to get debug bus defs
    bus_abstract_names = set(bus_abstract_names)
    dbus_defs = list(filter(
        lambda bd: bd.abstract_type.name in bus_abstract_names,
        bus_defs,
    ))
    
    pg, Z, wire_names = duhportinf._grouper.get_port_grouper(ports)
    pg_bus_pairings = duhportinf.main._get_bus_pairings(pg, dbus_defs)
    # filter only for port_group with dport
    pg_bus_pairings = list(filter(
        lambda x: dport in x[2],
        pg_bus_pairings,
    ))
    pg_bus_mappings = duhportinf.main._get_initial_bus_matches(pg, pg_bus_pairings)
    return list(map(lambda x: x[2:], pg_bus_mappings))
    

In [ ]:
port_path = '/ip-block-designs/tmp/tmp.json5' 
ports = duhportinf.get_ports_from_json5(port_path)
dport = ('front_port_axi4_0_ar_ready', 1, -1)
pg_bus_mappings = debug(ports, dport, ['AXI4_rtl'])

In [195]:
stime = time.time()

nvdla_pg_bus_mappings = duhportinf.get_bus_matches(nvdla_ports, list(bus_defs))
etime = time.time()
print('total time: {}s'.format(etime-stime))

total time: 2.5000040531158447s               


In [205]:
util.dump_json_bus_candidates('testy.json', nvdla_json_path, nvdla_pg_bus_mappings)

In [ ]:
stime = time.time()

ddr_pg_bus_mappings = duhportinf.get_bus_matches(ddr_ports, list(bus_defs))

etime = time.time()
print('total time: {}s'.format(etime-stime))

In [46]:
s_pcie_ports = list(filter(lambda p: 'axi4' not in p[0], pcie_ports))
stime = time.time()
pcie_pg_bus_mappings = duhportinf.get_bus_matches(s_pcie_ports, list(bus_defs))

etime = time.time()
print('total time: {}s'.format(etime-stime))

hierarchically clustering ports and selecting port groups
  - done
initial bus pairing with port groups
  - done
bus mapping
  - done                                                                         
total time: 13.784493684768677s


In [51]:
dport = ('pl_pll_ack', 1, 1)
for interface, bus_mappings in pcie_pg_bus_mappings:
    if dport in interface.ports:
        print('port group')
        for port in interface.ports:
            print('  - ', port)
        print('bus mappings')
        for bm in bus_mappings:
            print('  - cost:{} bus_def:{}'.format(
                bm.cost,
                str(bm.bus_def.abstract_type),
            ))

port group
  -  ('pl_rxdata', None, 1)
  -  ('pl_rxstatus', None, 1)
  -  ('pl_phystatus', None, 1)
  -  ('pl_rxstandby', None, -1)
  -  ('pl_rxpolarity', None, -1)
  -  ('pl_txcompliance', None, -1)
  -  ('pl_txelecidle', None, -1)
  -  ('pl_txsyncheader', None, -1)
  -  ('pl_txstartblock', None, -1)
  -  ('pl_txdatavalid', None, -1)
  -  ('pl_txdatak', None, -1)
  -  ('pl_txdata', None, -1)
  -  ('pl_txdetectrx', None, -1)
  -  ('pl_pll_rate', 3, -1)
  -  ('pl_pll_ack', 1, 1)
  -  ('pl_width', 2, -1)
  -  ('pl_ltssm', 5, -1)
  -  ('pl_blockaligncontrol', 1, -1)
  -  ('pl_txdeemph', 1, -1)
  -  ('pl_txswing', 1, -1)
  -  ('pl_txmargin', 3, -1)
  -  ('pl_rate', 2, -1)
  -  ('pl_powerdown', 2, -1)
  -  ('pl_spor', 1, 1)
  -  ('pl_npor', 1, 1)
  -  ('pl_srst', 1, 1)
  -  ('pl_rstnp', 1, 1)
  -  ('pl_rstn', 1, 1)
  -  ('pl_rstn_srst_out', 1, -1)
  -  ('pl_pclk_change_ok', None, 1)
  -  ('pl_pclk_change_ack', None, -1)
  -  ('pl_pclk', 1, 1)
  -  ('pl_pclk_rate', 3, -1)
bus mappings
  - co

In [ ]:
def dump_nonaxi_dendogram(path, ports):
    assert path.endswith('.pdf')
    nonaxi_ports = list(filter(
        lambda p: not p[0].startswith('axi'),
        ports,
    ))
    _, Z, wire_names = abxportinf.get_port_grouper(nonaxi_ports)
    fig, ax = pyplot.subplots(1,1, figsize=(10,40))
    _ = dendrogram(
        Z,
        ax=ax,
        orientation='left',
        labels=wire_names,
    )
    pyplot.savefig(path)


In [ ]:
dump_nonaxi_dendogram('ddr-nonaxi-dendogram.pdf', ddr_ports)
dump_nonaxi_dendogram('pcie-nonaxi-dendogram.pdf', pcie_ports)

In [ ]:
util.dump_json_bus_candidates('ddr-busprop.json', ddr_pg_bus_mappings)
util.dump_json_bus_candidates('pcie-busprop.json', pcie_pg_bus_mappings)